In [1]:
%load_ext autoreload
%autoreload 2
import warnings
warnings.simplefilter("ignore")

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import time
import matplotlib.pyplot as plt
import attacks
from tqdm import tqdm
from torchvision import datasets, transforms
import model

In [3]:
use_cuda = False
device = torch.device("cuda" if use_cuda else "cpu")
batch_size = 64

np.random.seed(42)
torch.manual_seed(42)


## Dataloaders
train_dataset = datasets.MNIST('./mnist_data/', train=True, download=True, transform=transforms.Compose(
    [transforms.ToTensor()]
))
test_dataset = datasets.MNIST('./mnist_data/', train=False, download=True, transform=transforms.Compose(
    [transforms.ToTensor(), transforms.Normalize((0.1307,), (0.3081,))]
))

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)


In [4]:
MAX_GRAD_NORM = 1.2
EPSILON = 50.0
DELTA = 1e-5
EPOCHS = 20

LR = 1e-3
MAX_PHYSICAL_BATCH_SIZE = 32

In [5]:
fc_model = model.fcNet()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(fc_model.parameters(), lr=LR)

In [8]:
from opacus.validators import ModuleValidator

errors = ModuleValidator.validate(fc_model, strict=False)
errors[-5:]

[]

In [8]:
def accuracy(preds, labels):
    return (preds == labels).mean()

In [9]:
from opacus import PrivacyEngine

privacy_engine = PrivacyEngine()

model, optimizer, train_loader = privacy_engine.make_private_with_epsilon(
    module=fc_model,
    optimizer=optimizer,
    data_loader=train_loader,
    epochs=EPOCHS,
    target_epsilon=EPSILON,
    target_delta=DELTA,
    max_grad_norm=MAX_GRAD_NORM,
)

print(f"Using sigma={optimizer.noise_multiplier} and C={MAX_GRAD_NORM}")